In [ ]:
# import dependencies
import os, torch, pandas as pd, numpy as np
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision.utils import make_grid
from torchvision import transforms, datasets, models
from torchvision.io import read_image
import matplotlib.pyplot as plt
import time, copy

plt.ion() 

In [ ]:
torch.cuda.is_available()

In [ ]:
# Config
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_path = '/kaggle/input/location-detect/hotel1_image_rest/train'
test_path = '/kaggle/input/location-detect/hotel1_image_rest/test'

In [ ]:
# Super Parameter Config
batch_size = 4
learning_rate = 1e-3
EPOCH = 20

In [ ]:
# Transforms
data_transforms = {
    'train': transforms.Compose([
        # cut anywhere by random.
        transforms.RandomResizedCrop(512),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# Load data
testdataset = datasets.ImageFolder(test_path, data_transforms['test'])
traindataset = datasets.ImageFolder(train_path, data_transforms['train'])
testdataset_loader = DataLoader(testdataset, batch_size=batch_size, shuffle=True)
traindataset_loader = DataLoader(traindataset, batch_size=batch_size, shuffle=True)

In [ ]:
# visulize img after transforming
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(traindataset_loader))

# Make a grid from batch
out = make_grid(inputs)

class_name = traindataset.classes
imshow(out, title=[class_name[x] for x in classes])

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        
        if batch % 2 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    scheduler.step()


def test_loop(dataloader, model, loss_fn, best_acc):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0.0, 0.0
    best_model_weight = copy.deepcopy(model.state_dict())

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            outputs = model(X)
            _, pred = torch.max(outputs,1)
            test_loss += loss_fn(outputs, y).item()
            correct += torch.sum(pred == y.data)

    test_loss /= num_batches
    correct /= size
    if(correct < best_acc):
        model.load_state_dict(best_model_weight)
        
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return (max(best_acc, correct), test_loss) 

In [ ]:
# load model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# 添加一层全连接层，将特征矩阵维数转为标签数
model.fc = nn.Linear(num_ftrs, 12)

model = model.to(device)


In [ ]:
0# start training
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
LR_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
best_correct = 0.0
LOSS = 0

for t in range(EPOCH):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(traindataset_loader, model, loss_fn, optimizer, LR_scheduler)
    (best_correct, LOSS) = test_loop(testdataset_loader, model, loss_fn, best_correct)
print("Done!")

In [ ]:
# Save model
print(f"LOSS:{LOSS}\nBEST_CORRECT:{best_correct}")
PATH = f"loss_{LOSS}_correct_{best_correct}.pt"
torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS
            }, PATH)